In [1]:
import numpy as np

In [2]:
# Set value of MISSING_INT here. In respy it is imported from shared_constants module.
MISSING_INT = -99

As a first step, we need to create the variables which would, in the final version, be supplied as impiuts to the function.

In [3]:
num_periods = 20
num_choices = 3 # Individuals can choose between full-time (F), part-time (P), and non-employment (N), i.e. there are 3 choices
educ_max = 25 # Maximum number of years of educ in the data, i.e., upper bound of education initial condition
educ_min = 10 # Minimim number of years of educ in the data, i.e., lower bound of education initial condition

In [4]:
# Auxiliary calculation of the education dimension
educ_range = educ_max - educ_min + 1

In [5]:
educ_range

16

Next, we create arrays which we want to populate with the state space variables.
The components of our state space are:
- the time variable: period
- the initial conditions: education
- the history of choices: choice_lagged

In [6]:
# Array for mapping the state space points to indices
shape = (num_periods, educ_range, num_choices)
mapping_state_index = np.tile(MISSING_INT, shape)

# Maximum number of state space points per period. There can be no more states in a period than this number. 
num_states_period_upper_bound = np.prod(mapping_state_index.shape)
print(mapping_state_index.shape, num_states_period_upper_bound)

(20, 16, 3) 960


Let us briefly discuss the dimension of the mapping_state_index array. Each dimension component corresponds to the state space component we would like to keep record of.
- dimension num_periods corresponds to the period number
- educ_range to the years of education
- num_choices to the number of choices available to the agents each period

In [7]:
# Array to collect all state space points (states) that can be reached each period
states_all = np.tile(MISSING_INT, (num_periods, num_states_period_upper_bound, 2))

Let us briefly discuss the dimention of the states_all array:
- the 1st dimension is determined by the number of periods in our model
- the 2nd dimension is related to the maximum number of state space points ever feasible / possibly reachable in one of the periods. *Question: Is this true? Is 100 000 simply some arbitrary large number that is for sure larger than the highest possible number of state space points ever reachable in a period? Can one replace this number by some educated guess of the maximum number of state point combinantions given no restrictions?*
- the 3rd dimension is equal to the number of remaining state space components (except period number) that we want to record education, choice_lagged

In [8]:
# Array for the maximum number state space points / states per period
states_number_period = np.tile(MISSING_INT, num_periods)

In a final step, we loop through all admissible state space points and fill up the constructed arrays with necessary information. Thereby, note two important details. 
- Since the individuals make their first choice in the first period, it is only possible to record their lagged choice from the second period onwards. Therefore, the loop directly skips the first period.
- If we want to record only admissible state space points, we have to account for the fact that individuals in the model start making labor supply choices only after they have completed education.

Let us look into latter in greater detail. As a reminder, we note that we model individuals from age 16 (legally binding mininum level of education) until age 60 (typical retirement entry age), i.e., for 45 periods, where the 1st period corresponds to age 16, the second period to age 17, etc. The current specification of starting values is equivalent to the observation / simulated reality that individuals in the sample have completed something between 10 and 25 years of edcuation. In our loop we want to take into account the fact that, in the first period at age 16, only individuals who have completed 10 years of education (assuming education for everyone starts at the age of 6) will be making a labor market choice between full-time (F), part-time (P), and non-employment (N). The remaining individuals are still in education, such that a state space point where years of education equal e.g. 11 and a labor market choice of e.g. part-time is observed is not admissible and should therefore not be recorded. This is ensured by the if clause "educ_years > period".

We also note that, in the current setting, there is no possibility of duplicate states arising.

In [10]:
# Loop over all periods / all ages
for period in range(num_periods)[1:]:
    
    # Start count for admissible state space points
    k = 0
        
    # Loop over all possible initial conditions for education
    for educ_years in range(educ_range):
            
        # Check if individual has already completed education
        # and will make a labor supply choice in the period
        if educ_years > period:
            continue
            
        # Loop over the three labor market choices, F, P, N
        for choice_lagged in [1,2,3]:
                
            # Check for duplicate states
            if (
                mapping_state_index[
                    period,
                    educ_years,
                    choice_lagged - 1,
                ]
                != MISSING_INT
            ):
                continue
                
            # Assign the integer count k as an indicator for the
            # currently reached admissible state space point
            mapping_state_index[
                period,
                educ_years,
                choice_lagged - 1,
            ] = k
                
            # Record the values of the state space components
            # for the currently reached admissible state space point
            states_all[period, k, :] = [
                educ_years + educ_min,
                choice_lagged -1,
            ]
                
            # Update count
            k += 1
    
    # Record number of admissible state space points for the period currently reached in the loop 
    states_number_period[period] = k

We briefly repeat what has been recorded in the states_all array:
 - educ_years + educ_min: in this example, values from 10 to 25
 - choice_lagged -1: 0,1,2 corresponding to F,P, and N

Note: Since there are no duplicate states in this setup, the check for duplicate states can be ommited.

In [11]:
# Auxiliary objects
max_states_period = max(states_number_period)

In [12]:
# Collect arguments
args = (states_all, states_number_period, mapping_state_index, max_states_period)

Questions and issues open for discussion:
1. MISSING_INT for number of states in the first period:
   The first open question conciders the handling of the first period (in code notation period == 0) in the model. For now the loop skips the first period entirely. Why? Since one of the state space components is a lagged value - choice_lagged - there is no value one can assign to this state space component in the first period. As individuals make their first choice in the first period there is no prior choice one can record as choice_lagged at the very beginning of the model.
   In respy the problem seams to not exist since the choices education and home seem to be valid entries of choice_lagged in the first period. Question: Where can this be deduced from in KW97, I do not remember coming across any explanation on this in the paper?

In [13]:
states_number_period

array([-99,   6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36,  39,
        42,  45,  48,  48,  48,  48,  48])

2. Education initial condition specification 

Ideas checks and tests:
- check if loop visits the whole range of values by checking min and max of array entries for the differnt state space components.

In [15]:
print(np.amax(states_all[:,:,0]), np.amax(states_all[:,:,1]))

25 2
